In [32]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [33]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [34]:
! nvidia-smi

Sun May 15 18:04:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   45C    P8    40W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [35]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
    # db_test1 = db.test_1
    # db_test2 = db.test_2
    # db_rm_us = db.rm_patent_us
    # db_rm_cn = db.rm_patent_cn
    # db_rm_de = db.rm_patent_de
    
    db_cn_5 = db.w_patent_cn_5
    db_cn_10 = db.w_patent_cn_10
    db_us_5 = db.w_patent_us_5
    db_us_10 = db.w_patent_us_10
    db_de_5 = db.w_patent_de_5
    db_de_10 = db.w_patent_de_10
except errors.ConnectionFailure as err:
    print(err)

In [36]:
db_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'patent_de')

In [19]:
test1 = pd.DataFrame(list(db_test1.find()))

In [20]:
test1.head()

,_id,appDate,priority
0,626d580bb9455959c69fc4b3,20100501,1
1,626d580bb9455959c69fc4b4,20100701,2
2,626d580bb9455959c69fc4b5,20100801,3
3,6272409c5477582d912a27fb,20100901,3


In [17]:
test2 = pd.DataFrame(db_test2.aggregate([{"$project":{"priority":1}}]))

In [18]:
test2

,_id,priority
0,626d576432510447df0abe89,1
1,626d576432510447df0abe8a,21
2,626d576432510447df0abe8b,3


In [21]:
a = test1.append(test2)

/tmp/ipykernel_31739/1407032937.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a = test1.append(test2)


In [22]:
a

,_id,appDate,priority
0,626d580bb9455959c69fc4b3,20100501,1
1,626d580bb9455959c69fc4b4,20100701,2
2,626d580bb9455959c69fc4b5,20100801,3
3,6272409c5477582d912a27fb,20100901,3
0,626d576432510447df0abe89,NaN,1
1,626d576432510447df0abe8a,NaN,21
2,626d576432510447df0abe8b,NaN,3


In [23]:
result = a.drop_duplicates(subset=['priority'],keep=False)

In [24]:
result

,_id,appDate,priority
1,626d580bb9455959c69fc4b4,20100701,2
1,626d576432510447df0abe8a,NaN,21


In [27]:
r = pd.concat([test1,test2,test2]).drop_duplicates(subset=['priority'],keep=False)

In [28]:
r

,_id,appDate,priority
1,626d580bb9455959c69fc4b4,20100701,2


# 切成前後五年存

## 前五年

### CN

In [6]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [7]:
db_cn_5 = pd.DataFrame(db_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [8]:
# 存進MongoDB
db.w_patent_cn_5.insert_many(db_cn_5.to_dict('records'))

### DE

In [9]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [10]:
db_de_5 = pd.DataFrame(db_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [11]:
# 存進MongoDB
db.w_patent_de_5.insert_many(db_de_5.to_dict('records'))

### US

In [12]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [13]:
db_us_5 = pd.DataFrame(db_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [14]:
# 存進MongoDB
db.w_patent_us_5.insert_many(db_us_5.to_dict('records'))

## 後五年

### CN

In [15]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [16]:
db_cn_10 = pd.DataFrame(db_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [17]:
# 存進MongoDB
db.w_patent_cn_10.insert_many(db_cn_10.to_dict('records'))

### DE

In [18]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [19]:
db_de_10 = pd.DataFrame(db_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [20]:
# 存進MongoDB
db.w_patent_de_10.insert_many(db_de_10.to_dict('records'))

### US

In [21]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [22]:
db_us_10 = pd.DataFrame(db_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [23]:
# 存進MongoDB
db.w_patent_us_10.insert_many(db_us_10.to_dict('records'))

# CPC計算

## 前五年

### DE

In [26]:
data_assignee = db_de_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [27]:
len(ls_company)

63

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_de_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_de = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_de

,company,cpc,count
0,Robert Bosch GmbH,12,9
1,Robert Bosch GmbH,H01L2224,8
2,Robert Bosch GmbH,H04L9,7
3,Robert Bosch GmbH,G08G1,7
4,Robert Bosch GmbH,F01N2610,7
...,...,...,...
1202,MOOREDOLL Inc,A63H2200,1
1203,MOOREDOLL Inc,36,1
1204,MOOREDOLL Inc,04,1
1205,MOOREDOLL Inc,28,1


In [29]:
df_cpc_de.shape

(1207, 3)

In [30]:
# 取cpc前3碼
df_cpc_de["cpc_n"] = df_cpc_de.cpc.apply(lambda x: x[0:3])

In [31]:
# df_cpc_de

In [32]:
# 轉成company-cpc 的總數表格
table_de = df_cpc_de.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [33]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A42,A61,A63,B05,B25,B27,B41,B42,B60,...,G09,G10,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,Accuragen Inc,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,1.0,0.0,0.0
4,Armstrong DLW GmbH,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
final_table_de.shape

(62, 36)

In [35]:
final_table_de.to_csv('w_company_cpc_de_5.csv',index=0)

In [36]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_de_5

In [37]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

In [38]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [37]:
data_assignee = db_cn_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [38]:
len(ls_company)

2077

In [39]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_cn_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [40]:
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [41]:
df_cpc.shape

(17823, 3)

In [42]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [43]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [44]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A24,A41,A42,A43,A44,A45,A46,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
final_table_cn.shape

(1006, 107)

In [30]:
final_table_cn.to_csv('w_company_cpc_cn_5.csv',index=0)

In [17]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_cn_5

In [18]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [19]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [52]:
data_assignee = db_us_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [53]:
len(ls_company)

705

In [54]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_us_5.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,266
1,Samsung Electronics Co Ltd,G06F3,252
2,Samsung Electronics Co Ltd,H01L27,126
3,Samsung Electronics Co Ltd,H04W52,119
4,Samsung Electronics Co Ltd,02,105


In [55]:
df_cpc_us.shape

(22211, 3)

In [56]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [57]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [58]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,8.0,0.0,3.0,4.0,0.0
3,AFI Licensing LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,AFIRMA CONSULTING & TECHNOLOGIES SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [59]:
final_table_us.shape

(700, 73)

In [60]:
final_table_us.to_csv('w_company_cpc_us_5.csv',index=0)

In [61]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_us_5

In [62]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [63]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## 後五年

### DE

In [64]:
data_assignee = db_de_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [65]:
len(ls_company)

390

In [66]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_de_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_de = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_de

,company,cpc,count
0,Intel Corp,G06F9,392
1,Intel Corp,H01L2224,228
2,Intel Corp,H04L67,156
3,Intel Corp,G06N3,152
4,Intel Corp,G06F12,134
...,...,...,...
13027,Kitz Corp,0642,1
13028,Kitz Corp,F16K5,1
13029,Kitz Corp,0025,1
13030,Kitz Corp,0283,1


In [67]:
df_cpc_de.shape

(13032, 3)

In [68]:
# 取cpc前3碼
df_cpc_de["cpc_n"] = df_cpc_de.cpc.apply(lambda x: x[0:3])

In [69]:
# df_cpc_de

In [70]:
# 轉成company-cpc 的總數表格
table_de = df_cpc_de.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [71]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A01,A24,A41,A42,A43,A44,A45,A47,A61,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
final_table_de.shape

(384, 80)

In [73]:
final_table_de.to_csv('w_company_cpc_de_10.csv',index=0)

In [74]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_de_10

In [75]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

In [76]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [77]:
data_assignee = db_cn_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [78]:
len(ls_company)

11480

In [79]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_cn_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,5511
1,Qualcomm Inc,H04W72,4354
2,Qualcomm Inc,H04L1,2475
3,Qualcomm Inc,H04B7,2160
4,Qualcomm Inc,H04L27,1105


In [80]:
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,5511
1,Qualcomm Inc,H04W72,4354
2,Qualcomm Inc,H04L1,2475
3,Qualcomm Inc,H04B7,2160
4,Qualcomm Inc,H04L27,1105


In [81]:
df_cpc.shape

(163336, 3)

In [82]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [83]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [84]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
1,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
2,10757616 Canada Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
3,10757616加拿大公司,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
4,"11th Street Co., Ltd.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,0.0


In [85]:
final_table_cn.shape

(7917, 124)

In [86]:
final_table_cn.to_csv('w_company_cpc_cn_10.csv',index=0)

In [87]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_cn_10

In [88]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [89]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [90]:
data_assignee = db_us_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [91]:
len(ls_company)

4957

In [92]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_us_10.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,8921
1,Qualcomm Inc,H04W72,7178
2,Qualcomm Inc,H04L1,3964
3,Qualcomm Inc,H04B7,3877
4,Qualcomm Inc,H04W52,2010


In [93]:
df_cpc_us.shape

(200793, 3)

In [94]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [95]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [96]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,12a Systems Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
final_table_us.shape

(4927, 119)

In [98]:
final_table_us.to_csv('w_company_cpc_us_10.csv',index=0)

In [99]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_cpc_us_10

In [100]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [101]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})